# Парсинг
Необходимо сформировать датасет со следующими данными:  
$\bullet$ Название фильма  
$\bullet$ Страна производства фильма  
$\bullet$ Жанр фильма  
$\bullet$ Главные актеры фильма  
$\bullet$ Режиссер фильма  
$\bullet$ Длительность фильма в минутах  
$\bullet$ Описание фильма  
$\bullet$ Оценка фильма  

Подключение необходимых библиотек

In [2]:
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
import csv
from random import uniform
from time import sleep

In [3]:
option = Options()
option.add_argument("--disable-infobars")
browser = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()),
                          options=option)

Парсинг 20 вкладок страницы «Популярные фильмы» для получения 1000 ссылок на страницы фильмов

In [12]:
base_url = 'https://www.kinopoisk.ru/lists/movies/popular-films/?page={}'
pages = []
urls = [base_url.format(x) for x in [int(num) for num in 
                                     np.linspace(1, 20, 20)]]
for url in urls:
    browser.get(url)
    soup = bs(browser.page_source, 'lxml')
    page_urls = [page['href'] for page in 
                 soup.find_all('a', {'class': 'base-movie-main-info_link__YwtP1'})]
    pages.extend(page_urls)

Форматирование полученных ссылок

In [13]:
links = ['https://www.kinopoisk.ru' + page for page in pages] 

Проверка количества полученных ссылок

In [15]:
len(links)

1000

Создание структуры для последующего заполнения данными

In [39]:
data = {
    'title': [], # Название фильма
    'country': [], # Страна производства фильма
    'genre': [], # Жанр фильма
    'actors': [], # Главные актеры фильма
    'director': [], # Режиссер фильма
    'time': [], # Длительность фильма в минутах
    'description': [], # Описание фильма
    'score': [] # Оценка фильма
}

Парсинг страниц фильмов по полученным ссылкам и заполнение структуры данными

In [40]:
for link in links:
    # Парсинг страницы фильма
    browser.get(link)
    soup = bs(browser.page_source, 'html')
    
    try:
        # Название фильма
        title = soup.find('span', {'data-tid': '75209b22'}).text.split("(")[0]
        
        # Страна производства фильма
        country = [c.find('a').text for c in 
                   soup.find_all('div', {'data-tid': 'd5ff4cc'}) 
                   if c.find('a')['href'].startswith("/lists/movies/country") 
                   == True]
        
        # Жанр фильма
        genre = [g.text for g in 
                 soup.find('div', {'data-tid': '28726596'}).find_all('a')]
        
        # Главные актеры фильма
        actors = [act.text for act in 
                  soup.find('ul', {'class': 'styles_list___ufg4'}).find_all('a')[0:3]]
        
        # Режиссер фильма
        director = [a.text for a in 
                    soup.find_all('a', {'data-tid': '603f73a4'}) 
                    if a['href'].startswith('/name')][0]
        
        # Описание фильма
        description = str(soup.find('p', {'class': 'styles_paragraph__wEGPz'}).text).replace('\xa0', ' ')
        
        # Длительность фильма в минутах
        time = [t.text.split(' ')[0] for t in soup.find_all('div', {'data-tid': 'e1e37c21'}) 
                if (len(t.text.split(' ')) > 1 and t.text.split(' ')[1] == 'мин.')][0]
        
        # Оценка фильма
        score = float(soup.find('span', {'data-tid': '7f8f8841'}).text.split(' ')[2])
        
    except:
        # Пропуск неподходящих страниц
        continue
    
    # Добавление записей
    data['title'] += [title]
    data['country'] += [country]
    data['genre'] += [genre]
    data['actors'] += [actors]
    data['director'] += [director]
    data['description'] += [description]
    data['time'] += [time]
    data['score'] += [score]
    
    # Случайная пауза между парсингом страниц
    sleep(uniform(1, 15))

Создание датафрейма на основе заполненной структуры

In [42]:
dt = pd.DataFrame(data = data)

Проверка наличия в датафрейме пропусков данных

In [53]:
dt.isnull().sum()

title          0
country        0
genre          0
actors         0
director       0
time           0
description    0
score          0
dtype: int64

Вывод первых 10 строк

In [62]:
dt.head(10)

,title,country,genre,actors,director,time,description,score
0,Бременские музыканты,[Россия],"[приключения, семейный, фэнтези, комедия]","[Тихон Жизневский, Валентина Ляпина, Роман Кур...",Алексей Нужный,116,"Трубадур и его друзья-самозванцы — Пес, Кошка,...",7.1
1,По щучьему велению,[Россия],"[фэнтези, приключения, семейный]","[Никита Кологривый, Мила Ершова, Алина Алексеева]",Александр Войтинский,115,"Если ты идешь на рыбалку — будь готов к тому, ...",7.8
2,Иван Васильевич меняет всё,[Россия],"[комедия, фантастика, приключения, слова]","[Тимур Батрутдинов, Демис Карибидис, Максим Ла...",Миша Семичев,106,Шурик пытается с помощью машины времени вернут...,6.6
3,Триггер. Фильм,[Россия],[драма],"[Максим Матвеев, Ирина Старшенбаум, Виктория М...",Александра Ремизова,99,После драматичных событий — бывшая жена осужде...,6.0
4,Три богатыря и Пуп Земли,[Россия],"[мультфильм, приключения, семейный]","[Олег Куликович, Валерий Соловьев, Дмитрий Бык...",Константин Феоктистов,85,"По сказкам мы знаем, что было давным-давно, но...",5.6
5,Холоп,[Россия],"[комедия, мелодрама]","[Милош Бикович, Александра Бортич, Александр С...",Клим Шипенко,109,27-летний московский мажор Григорий ошалел от ...,7.0
6,Мальчик и птица,[Япония],"[аниме, мультфильм, драма, фэнтези, слова]","[Сома Сантоки, Аимён, Кэйко Такэсита]",Хаяо Миядзаки,124,Мальчик Махито сильно тоскует по своей матери ...,7.7
7,Тёща,[Россия],[комедия],"[Лариса Гузеева, Гарик Харламов, Настасья Самб...",Аскар Узабаев,98,Ольга Николаевна — волевая женщина и начальниц...,6.0
8,Оппенгеймер,[США],"[биография, драма, история, слова]","[Киллиан Мерфи, Эмили Блант, Мэтт Дэймон]",Кристофер Нолан,180,История жизни американского физика-теоретика Р...,8.2
9,Убийцы цветочной луны,[США],"[драма, криминал, история, слова]","[Леонардо ДиКаприо, Роберт Де Ниро, Лили Гладс...",Мартин Скорсезе,206,"Вскоре после Первой мировой Эрнест Беркхарт, о...",7.2


Сохранение датафрейма в формате CSV

In [67]:
dt.to_csv('./datasets/data.csv')

### Создание обучающей и тестовой выборок
Разделим датасет на обучающую и тестовую выборку в отношении 7:3. Каждую выборку сохраним в формате CSV

Обучающая выборка

In [68]:
train = dt.head(int(len(dt) * 0.7))
train.to_csv('./datasets/train.csv')

Тестовая выборка

In [69]:
test = dt.tail(int(len(dt) * 0.3) + 1)
test.to_csv('./datasets/test.csv')